In [2]:
import io
import os
import json
import requests
import chess.pgn
import pandas as pd
from tqdm import tqdm

DB_PATH = '/home/miquel/proyectosPython/JUEGOS/ajedrez/chess_engine/db'
HEADERS = {
    'User-Agent': 'MyChessApp/1.0'
}

In [3]:
def fetch_archives(username):
    archives_url = f'https://api.chess.com/pub/player/{username}/games/archives'
    response = requests.get(archives_url, headers=HEADERS)
    if response.status_code == 200:
        return response.json().get('archives', [])
    else:
        print(f"Failed to fetch archives: \n{response}")
        return []


def fetch_games(archive_url):
    response = requests.get(archive_url, headers=HEADERS)
    if response.status_code == 200:
        return response.json().get('games', [])
    else:
        print(f"Failed to fetch games from {archive_url}: {response.status_code}")
        return []


def fetch_all_games(username):
    """Fetches all games for a given Chess.com username."""
    archives = fetch_archives(username)
    all_games = []
    for archive_url in archives:
        print(f"Fetching games from {archive_url}...")
        games = fetch_games(archive_url)
        all_games.extend(games)
    
    for i, game in tqdm(enumerate(all_games),desc="Parsing moves", total=len(games)):
        sans, origins, destinies = parse_pgn(game['pgn'])
        all_games[i]['sans'] = sans
        all_games[i]['origins'] = origins
        all_games[i]['destinies'] = destinies
        
    return all_games


def parse_pgn(pgn_string):
    pgn = io.StringIO(pgn_string)
    game = chess.pgn.read_game(pgn)
    board = game.board() # to get the origin - destiny format we need to simulate the game
    origins = []
    destinies = []
    sans = []
    for move in game.mainline_moves():
        sans.append(board.san(move))
        origins.append(chess.square_name(move.from_square))
        destinies.append(chess.square_name(move.to_square))
        
        board.push(move)  # Update the board with the move
    
    return sans, origins, destinies


def save_games_to_json(username, games):
    filepath = os.path.join(DB_PATH, f'{username}_games.json')
    with open(filepath, 'w') as f:
        json.dump(games, f, indent=4)
    print(f"Saved {len(games)} games to {filepath}")

In [6]:
# username = 'miquelllorca'
# username = 'naviaras'
# username = 'naviaras00'
username = 'navipb00'
# username = 'imnotalolplayer' # lichess
all_games = fetch_all_games(username)
save_games_to_json(username, all_games)

Fetching games from https://api.chess.com/pub/player/navipb00/games/2024/04...
Fetching games from https://api.chess.com/pub/player/navipb00/games/2024/05...


Parsing moves: 145it [00:00, 281.11it/s]             


Saved 145 games to /home/miquel/proyectosPython/JUEGOS/ajedrez/chess_engine/db/navipb00_games.json


In [5]:
all_games[0]

{'url': 'https://www.chess.com/game/live/4773442624',
 'pgn': '[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2020.04.26"]\n[Round "-"]\n[White "miquelllorca"]\n[Black "henry14605"]\n[Result "0-1"]\n[CurrentPosition "r1bqkb1r/pppp1ppp/8/3Pp3/3n4/8/PPPP1PPP/R1BQKBNR w KQkq -"]\n[Timezone "UTC"]\n[ECO "A00"]\n[ECOUrl "https://www.chess.com/openings/Van-Geet-Opening-Reversed-Nimzowitsch-Variation-2.e3"]\n[UTCDate "2020.04.26"]\n[UTCTime "10:25:36"]\n[WhiteElo "845"]\n[BlackElo "1049"]\n[TimeControl "60"]\n[Termination "henry14605 won on time"]\n[StartTime "10:25:36"]\n[EndDate "2020.04.26"]\n[EndTime "10:26:43"]\n[Link "https://www.chess.com/game/live/4773442624"]\n\n1. e3 {[%clk 0:00:46.5]} 1... e5 {[%clk 0:00:58.9]} 2. Nc3 {[%clk 0:00:42]} 2... Nc6 {[%clk 0:00:58.5]} 3. e4 {[%clk 0:00:24.4]} 3... Nf6 {[%clk 0:00:57.9]} 4. Nd5 {[%clk 0:00:08.6]} 4... Nxd5 {[%clk 0:00:56.3]} 5. exd5 {[%clk 0:00:03.7]} 5... Nd4 {[%clk 0:00:55.7]} 0-1\n',
 'time_control': '60',
 'end_time': 1587896803,
 '

In [9]:
filtered_games = pd.DataFrame(all_games)
print(filtered_games.columns)
# filtered_games

Index(['url', 'pgn', 'time_control', 'end_time', 'rated', 'tcn', 'uuid',
       'initial_setup', 'fen', 'time_class', 'rules', 'white', 'black', 'sans',
       'origins', 'destinies', 'accuracies', 'start_time', 'tournament'],
      dtype='object')


In [ ]:
# filtra por ELO, por ejemplo


In [10]:
filtered_games = filtered_games.drop(columns=['url','pgn','end_time','rated', 'tcn', 'uuid', 'initial_setup', 'fen', 'rules', 'start_time', 'tournament'])

,time_control,fen,time_class,rules,white,black,sans,origins,destinies,accuracies,start_time,tournament
0,60,r1bqkb1r/pppp1ppp/8/3Pp3/3n4/8/PPPP1PPP/R1BQKB...,bullet,chess,"{'rating': 845, 'result': 'timeout', '@id': 'h...","{'rating': 1049, 'result': 'win', '@id': 'http...","[e3, e5, Nc3, Nc6, e4, Nf6, Nd5, Nxd5, exd5, Nd4]","[e2, e7, b1, b8, e3, g8, c3, f6, e4, c6]","[e3, e5, c3, c6, e4, f6, d5, d5, d5, d4]",NaN,NaN,NaN
1,600,2k2Q2/1p6/p4p1p/8/1P6/P4P2/2P3PP/R3K1NR b KQ -,blitz,chess,"{'rating': 531, 'result': 'win', '@id': 'https...","{'rating': 473, 'result': 'resigned', '@id': '...","[e4, Nf6, d3, d6, Nc3, c5, d4, cxd4, Qxd4, Ng4...","[e2, g8, d2, d7, b1, c7, d3, c5, d1, f6, f1, c...","[e4, f6, d3, d6, c3, c5, d4, d4, d4, g4, b5, d...",NaN,NaN,NaN
2,600,Q7/8/2pk4/3p1r2/1p6/4PB2/1P2KPP1/8 w - -,blitz,chess,"{'rating': 336, 'result': 'timeout', '@id': 'h...","{'rating': 530, 'result': 'win', '@id': 'https...","[e3, Nf6, d4, Nc6, Nc3, e6, Nb5, Bb4+, c3, a6,...","[e2, g8, d2, b8, b1, e7, c3, f8, c2, a7, c3, a...","[e3, f6, d4, c6, c3, e6, b5, b4, c3, a6, b4, b...",NaN,NaN,NaN
3,1800,r1b1kb1r/pp3pp1/n1p1p3/3p4/3P4/2N1P2P/PPP1B1q1...,rapid,chess,"{'rating': 693, 'result': 'resigned', '@id': '...","{'rating': 1107, 'result': 'win', '@id': 'http...","[d3, c6, Nc3, d6, d4, d5, e3, Nf6, Nf3, Na6, B...","[d2, c7, b1, d7, d3, d6, e2, g8, g1, b8, f1, e...","[d3, c6, c3, d6, d4, d5, e3, f6, f3, a6, d3, e...","{'white': 7.808119899585552, 'black': 37.94786...",NaN,NaN
4,600,rn2k2r/1Q3ppp/p3p2n/2b5/4P1b1/3P4/PPP2qPP/R1B1...,blitz,chess,"{'rating': 275, 'result': 'checkmated', '@id':...","{'rating': 526, 'result': 'win', '@id': 'https...","[e4, d5, d3, Nh6, Nc3, Bg4, Qd2, d4, Nb5, a6, ...","[e2, d7, d2, g8, b1, c8, d1, d5, c3, a7, b5, d...","[e4, d5, d3, h6, c3, g4, d2, d4, b5, a6, d4, d...","{'white': 4.455042397543517, 'black': 34.23699...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
440,1/86400,rnbqkbnr/ppp2ppp/4p3/3p4/8/1P4P1/P1PPPPBP/RNBQ...,daily,chess,"{'rating': 800, 'result': 'win', '@id': 'https...","{'rating': 845, 'result': 'timeout', '@id': 'h...","[g3, d5, Bg2, e6, b3]","[g2, d7, f1, e7, b2]","[g3, d5, g2, e6, b3]","{'white': 96.66, 'black': 100}",1.714721e+09,NaN
441,1/86400,6k1/p4r1p/7K/7P/b3p1p1/5p2/1n6/8 w - - 0 43,daily,chess,"{'rating': 835, 'result': 'resigned', '@id': '...","{'rating': 1032, 'result': 'win', '@id': 'http...","[e4, c5, c3, d6, d4, cxd4, cxd4, Nf6, Qa4+, Bd...","[e2, c7, c2, d7, d2, c5, c3, g8, d1, c8, f1, f...","[e4, c5, c3, d6, d4, d4, d4, f6, a4, d7, b5, e...","{'white': 69.54, 'black': 83.05}",1.714658e+09,NaN
442,1800,2r5/p6p/3p4/3B4/1PbP2Qk/P3PR1P/1K6/8 b - -,rapid,chess,"{'rating': 1479, 'result': 'win', '@id': 'http...","{'rating': 773, 'result': 'checkmated', '@id':...","[a3, d6, h3, Nf6, b4, b5, Bb2, Nc6, Nf3, e5, d...","[a2, d7, h2, g8, b2, b7, c1, b8, g1, e7, d2, d...","[a3, d6, h3, f6, b4, b5, b2, c6, f3, e5, d3, d...",NaN,NaN,NaN
443,1/86400,r1bk4/1pp5/3p2N1/p5n1/P7/2P2N2/1P2BPPP/R1B1R1K...,daily,chess,"{'rating': 1065, 'result': 'win', '@id': 'http...","{'rating': 825, 'result': 'resigned', '@id': '...","[a4, a5, e4, e5, d4, exd4, Qxd4, Bb4+, c3, Be7...","[a2, a7, e2, e7, d2, e5, d1, f8, c2, b4, d4, e...","[a4, a5, e4, e5, d4, d4, d4, b4, c3, e7, g7, f...",NaN,1.715773e+09,NaN
